In [1]:
include("nn_ops.jl")

NNOps

In [34]:
batch = 1
in_height = 28
in_width = 28
stride_height = 2
stride_width = 2
pooled_height = round(Int, in_height/stride_height, RoundUp)
pooled_width = round(Int, in_width/stride_width, RoundUp)
in_channels = 1
filter_height = 5
filter_width = 5
out_channels = 32

srand(5)
x_actual = rand(-10:10, batch, in_height, in_width, in_channels)
x_current = rand(-10:10, batch, in_height, in_width, in_channels)
filter = rand(-10:10, filter_height, filter_width, in_channels, out_channels)
x_conv_actual = NNOps.conv2d(x_actual, filter)
x_conv_relu_actual = NNOps.relu(x_conv_actual)
x_conv_relu_maxpool_actual = NNOps.maxpool(x_conv_relu_actual, (1, 2, 2, 1));

In [35]:
using JuMP
using Gurobi

m = Model(solver=GurobiSolver())

@variable(m, ve[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m, vx[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m, vx_conv[1:batch, 1:in_height, 1:in_width, 1:out_channels])
# @variable(m, vx_conv_relu[1:batch, 1:in_height, 1:in_width, 1:out_channels])
# @variable(m, vx_conv_relu_maxpool[1:batch, 1:pooled_height, 1:pooled_width, 1:out_channels])
@constraint(m, vx .== x_current) # input

# 1. Only using convolution constraint
vx_conv = NNOps.conv2dconstraint(m, vx+ve, filter)
# @constraint(m, conv2d(vx+ve, filter) .== vx_conv)
# @constraint(m, vx_conv .== x_conv_actual)

# 2. Adding relu layer
vx_conv_relu = NNOps.reluconstraint(m, vx_conv, 10000)
# @constraint(m, vx_conv_relu .== x_conv_relu_actual)

# 3. Adding maxpool layer
vx_conv_relu_maxpool = NNOps.maxpoolconstraint(m, vx_conv_relu, (2, 2), 10000)
@constraint(m, vx_conv_relu_maxpool .== x_conv_relu_maxpool_actual)


@objective(m, Min, sum(ve.^2))

:Min

In [ ]:
status = solve(m)

println("Objective value: ", getobjectivevalue(m))
# TODO: Are jump solutions global? Can I save particular variables?
println("e = ", getvalue(ve))

# TODO: fix the error message WARNING: A variable named a is already attached to this model. If creating variables programmatically, consider using the anonymous variable syntax x = @variable(m, [1:N], ...).

In [30]:
x_conv_current = NNOps.conv2d(x_current+getvalue(ve), filter)
x_conv_relu_current = NNOps.relu(x_conv_current)
x_conv_relu_maxpool_current = NNOps.maxpool(x_conv_relu_current, (1, 2, 2, 1));

In [31]:
maximum(abs(x_conv_current - x_conv_actual))

2093.3744208028265

In [32]:
maximum(abs(x_conv_relu_current - x_conv_relu_actual))

1452.0

In [33]:
maximum(abs(x_conv_relu_maxpool_current - x_conv_relu_maxpool_actual))

1.3869794202037156e-11

In [11]:
maximum(abs(x_conv_current))

338.7638695133706

In [ ]:
maximum(abs(x_conv_current))